In [1]:
%reload_ext autoreload
%autoreload 2
%reload_ext lab_black

In [2]:
epoch_key = "bon", 3, 2
ripple_number = 1

In [161]:
import xarray as xr
import os
import numpy as np
from src.analysis import get_probability, get_is_classified
from src.parameters import PROCESSED_DATA_DIR

animal, day, epoch = epoch_key
data_type, dim = "clusterless", "1D"

results = xr.open_dataset(
    os.path.join(PROCESSED_DATA_DIR, f"{animal}_{day:02}_{epoch:02}.nc"),
    group=f"/{data_type}/{dim}/classifier/ripples/",
)

probability_threshold = 1.0

probability = get_probability(results)
is_classified = get_is_classified(probability, probability_threshold)
is_classified.sum(("ripple_number", "time"))

<xarray.DataArray 'is_classified' (state: 5)>
array([2436., 8308.,  529.,    0.,    0.])
Coordinates:
  * state    (state) object 'Continuous' ... 'Fragmented-Continuous-Mix'

In [149]:
is_classified.sum(("ripple_number", "time"))

<xarray.DataArray 'is_classified' (state: 5)>
array([2436., 8308.,  529.,    0.,    0.])
Coordinates:
  * state    (state) object 'Continuous' ... 'Fragmented-Continuous-Mix'

In [36]:
probability.sel(state=["Hover", "Continuous", "Fragmented"])

<xarray.DataArray 'acausal_posterior' (ripple_number: 251, time: 127, state: 3)>
array([[[9.84018046e-01, 1.59476563e-02, 3.42979981e-05],
        [9.83850903e-01, 1.61196427e-02, 2.94547295e-05],
        [9.83705394e-01, 1.62726984e-02, 2.19076336e-05],
        ...,
        [           nan,            nan,            nan],
        [           nan,            nan,            nan],
        [           nan,            nan,            nan]],

       [[9.26526793e-01, 7.34559593e-02, 1.72472604e-05],
        [9.27076450e-01, 7.29171528e-02, 6.39749908e-06],
        [9.27664333e-01, 7.23248730e-02, 1.07938582e-05],
        ...,
        [           nan,            nan,            nan],
        [           nan,            nan,            nan],
        [           nan,            nan,            nan]],

       [[6.70208580e-01, 3.29716068e-01, 7.53520242e-05],
        [6.63334555e-01, 3.36574414e-01, 9.10313678e-05],
        [6.56557081e-01, 3.43368765e-01, 7.41543853e-05],
        ...,
...
        ...,
        [           nan,            nan,            nan],
        [           nan,            nan,            nan],
        [           nan,            nan,            nan]],

       [[4.82288560e-01, 5.14505123e-01, 3.20631739e-03],
        [4.85221815e-01, 5.12186314e-01, 2.59187087e-03],
        [4.88172122e-01, 5.09944583e-01, 1.88329547e-03],
        ...,
        [           nan,            nan,            nan],
        [           nan,            nan,            nan],
        [           nan,            nan,            nan]],

       [[1.97258649e-01, 5.01709295e-01, 3.01032056e-01],
        [2.04389458e-01, 5.54356445e-01, 2.41254097e-01],
        [2.13653268e-01, 6.08297930e-01, 1.78048802e-01],
        ...,
        [           nan,            nan,            nan],
        [           nan,            nan,            nan],
        [           nan,            nan,            nan]]])
Coordinates:
  * time           (time) timedelta64[ns] 00:00:00 ... 00:00:00.252000
  * ripple_number  (ripple_number) int64 1 2 3 4 5 6 ... 246 247 248 249 250 251
  * state          (state) object 'Hover' 'Continuous' 'Fragmented'

In [112]:
is_classified = ((probability.copy() * 0.0).fillna(False)).astype(bool)
blah = np.max(
    probability.sel(state=["Hover", "Continuous", "Fragmented"]).values, axis=-1
)
blah

TypeError: _argmax_dispatcher() got an unexpected keyword argument 'keepdims'

<xarray.DataArray 'acausal_posterior' (ripple_number: 251, time: 127)>
array([[ 1.,  1.,  1., ..., nan, nan, nan],
       [ 1.,  1.,  1., ..., nan, nan, nan],
       [ 1.,  1.,  1., ..., nan, nan, nan],
       ...,
       [ 1.,  1.,  1., ..., nan, nan, nan],
       [ 1.,  1.,  1., ..., nan, nan, nan],
       [ 1.,  1.,  1., ..., nan, nan, nan]])
Coordinates:
  * time           (time) timedelta64[ns] 00:00:00 ... 00:00:00.252000
  * ripple_number  (ripple_number) int64 1 2 3 4 5 6 ... 246 247 248 249 250 251

In [111]:
blah.shape, is_classified.shape

((251, 127), (251, 127, 5))

In [116]:
is_classified.values[blah[..., None]] = True

In [142]:
is_classified = ((probability.copy() * 0.0).fillna(False)).astype(bool)
A = probability.sel(state=["Hover", "Continuous", "Fragmented"]).values
A = A.argmax(axis=-1)[..., None] == np.arange(A.shape[-1])
A = np.concatenate((A, np.zeros((*A.shape[:2], 2), dtype=bool)), axis=-1)
is_classified.values = A
is_classified = is_classified * probability.sel(
    state=["Hover", "Continuous", "Fragmented"]
).sum("state", skipna=False)
is_classified

<xarray.DataArray 'acausal_posterior' (ripple_number: 251, time: 127, state: 5)>
array([[[ 1.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.],
        ...,
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]],

       [[ 1.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.],
        ...,
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]],

       [[ 1.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.],
        [ 1.,  0.,  0.,  0.,  0.],
        ...,
...
        ...,
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]],

       [[ 0.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.],
        ...,
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]],

       [[ 0.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.,  0.],
        ...,
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]]])
Coordinates:
  * time           (time) timedelta64[ns] 00:00:00 ... 00:00:00.252000
  * ripple_number  (ripple_number) int64 1 2 3 4 5 6 ... 246 247 248 249 250 251
  * state          (state) object 'Continuous' ... 'Fragmented-Continuous-Mix'